<a href="https://colab.research.google.com/github/jerry26432341/myTheses/blob/main/0524meeting_%E8%88%8A%E6%95%B8%E6%93%9A_%E6%8F%9B%E9%A0%90%E8%99%95%E7%90%86%E6%96%B9%E5%BC%8F_t%2Bd_92_7_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy
!python -m spacy download zh_core_web_sm
!wget -O chineseStopWords.txt https://raw.githubusercontent.com/tomlinNTUB/Machine-Learning-Basics/main/Chinese%20Stopwords.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 26.0 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2
2023-05-20 09:30:57.189076: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 09:30:58.539223: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 3

In [ ]:
import pandas as pd
import numpy as np
import jieba
import re
import gensim
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, vstack
from sklearn.svm import LinearSVC
from joblib import dump
from sklearn.model_selection import train_test_split
from google.colab import drive

# 掛載Google Drive
drive.mount('/content/drive')

# 读取数据
df = pd.read_csv('/content/drive/MyDrive/shopping.csv')

# 載入中文模型
nlp = spacy.load('zh_core_web_sm')

# 載入停用詞列表
with open('chineseStopWords.txt', 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f.readlines()]




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print("在 cat 列中总共有 %d 个空值." % df['cat'].isnull().sum())
print("在 review 列中总共有 %d 个空值." % df['review'].isnull().sum())
df[df.isnull().values==True]
df = df[pd.notnull(df['review'])]

在 cat 列中总共有 0 个空值.
在 review 列中总共有 1 个空值.


In [ ]:
df['cat_id'] = df['cat'].factorize()[0]
cat_id_df = df[['cat', 'cat_id']].drop_duplicates().sort_values('cat_id').reset_index(drop=True)
cat_to_id = dict(cat_id_df.values)
id_to_cat = dict(cat_id_df[['cat_id', 'cat']].values)
df.sample(10)

,cat,label,review,cat_id
15492,手机,0,相比索爱同端手机T618 T628却缺少红外线，而且蓝牙连接繁琐（输和入有两个程序）。导航键...,2
35866,洗发水,0,绝对的差评，漏了都没有办法投诉，礼品盒不拆开又不知道漏不漏，拆开了有不给退！也是醉了！终于知...,4
27000,洗发水,1,很好用 一直在用这个 洗完头发非常柔顺,4
5981,平板,1,还不错，用着还算不错，就是华为平板的配件不多啊,1
5304,平板,1,使用体验较为流畅，双4G版本配合流量卡，可以满足大多数需求，并且还可以作为移动WiFi，是主...,1
3212,书籍,0,看完了，书印的挺精致，但是内容：泡菜女人的喃喃自语，很一般啦！这本书总体感觉“见面不如闻名”...,0
58535,酒店,0,窗帘破旧，早上阳光都遮不住，地毯起码几十年破旧又脏，房间装修都很老，早餐东西只上一次，从不续...,9
6995,平板,1,还可以。。。。。。。。。。,1
1213,书籍,1,买这本书时正是工作超累之后，身体很不好的时候，消炎药和退烧药去火药是常服的，但是直接一时之痛...,0
7604,平板,1,不错不错！一直在用华为的！手机也是！,1


In [ ]:
# 定義停用詞過濾器
def filter_stopwords(text):
    tokens = []
    for token in nlp(text):
        if not token.is_stop and not token.is_punct and not token.is_space and not token.is_digit:
            if token.text not in stopwords:
                tokens.append(token.text)
    return ' '.join(tokens)

In [ ]:
# 新增一個 'filtered_review' 欄位，儲存過濾後的文本
df['filtered_review'] = df['review'].apply(filter_stopwords)

# 加载TF-IDF模型和Doc2Vec模型
tfidf = TfidfVectorizer()
tfidf_vectors = tfidf.fit_transform(df['filtered_review'])

d2v_model = gensim.models.Doc2Vec.load('/content/drive/MyDrive/doc2vec_model')

# 使用Doc2Vec模型对预处理后的文本进行向量化
d2v_vectors = []
for text in df['filtered_review']:
    vec = d2v_model.infer_vector(text.split())
    d2v_vectors.append(vec)
d2v_vectors = np.array(d2v_vectors)

# 使用hstack将TF-IDF向量和Doc2Vec向量连接起来
feature_matrix = hstack((tfidf_vectors, d2v_vectors))

# 定义训练数据
X_train = feature_matrix

# 定义标签
y_train = df['cat']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, df['cat'], test_size=0.2, random_state=42, shuffle=True)

# 训练LinearSVC模型
model = LinearSVC()
model.fit(X_train, y_train)

# 进行预测
predictions = model.predict(X_test)

# 重置X_test的索引并创建一个新的DataFrame
X_test_df = pd.DataFrame(X_test.toarray())
X_test_df.reset_index(drop=True, inplace=True)

# 创建一个新的数据框存储预测结果
df_predicted = pd.DataFrame({'review': df['review'].values[X_test_df.index], 'predicted_category': predictions})

# 输出预测结果
print(df_predicted)

                                                  review predicted_category
0      ﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持...                 平板
1      作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...                洗发水
2      作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...                 平板
3      作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...                 水果
4      作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...                计算机
...                                                  ...                ...
12550                                 直接不能用，刚来点没反应，屏幕不能用                 酒店
12551            收到了，有点后悔，除了硬件升级之外，外观比x1丑太多了，无比丑的大黑边，崩溃中                 蒙牛
12552                                           一般，还没给赠品                 酒店
12553  京东描述带手机卡的分辨率是1920×1080，结果只有1280×800，多花这些钱不是为了能...                 衣服
12554                                   刚拿到手 内存一半就没了 太坑了                 蒙牛

[12555 rows x 2 columns]


In [ ]:
from google.colab import files
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# 定义测试数据
X_test = feature_matrix

# 进行预测
predictions = model.predict(X_test)

# 定义测试标签
y_test = df['cat']

# 计算准确率
accuracy = accuracy_score(y_test, predictions)

# 计算召回率
recall = recall_score(y_test, predictions, average='weighted')

# 计算精确率
precision = precision_score(y_test, predictions, average='weighted')

# 计算F1分数
f1 = f1_score(y_test, predictions, average='weighted')

# 将X_test转换为密集矩阵
X_test_dense = X_test.toarray()

# 创建包含预测结果的 DataFrame，仅使用测试数据集的索引
df_results = pd.DataFrame({'Review': df['review'].values[X_test_df.index], 'Predicted_Category': predictions, 'Actual_Category': y_test})

# 将预测结果分为预测正确和预测错误的两个数据框，仅使用测试数据集的索引
correct_predictions = df_results[df_results['Predicted_Category'] == df_results['Actual_Category']]
incorrect_predictions = df_results[df_results['Predicted_Category'] != df_results['Actual_Category']]

# 将预测正确和预测错误的结果保存为 Excel 文件
correct_predictions.to_excel('correct_predictions.xlsx', index=False)
incorrect_predictions.to_excel('incorrect_predictions.xlsx', index=False)

# 下载文件到本地
files.download('correct_predictions.xlsx')
files.download('incorrect_predictions.xlsx')

# 输出指标结果
print("准确率:", accuracy)
print("召回率:", recall)
print("精确率:", precision)
print("F1分数:", f1)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

准确率: 0.9272298599716439
召回率: 0.9272298599716439
精确率: 0.9289564451390269
F1分数: 0.9275376028507634
